# Create Merged Dataset

In this workbook, you will read in the `trip` and `fare` files. You are welcome to use DataFrame and/or SparkSQL API as you desire as long as it produces the expected results.

## Instructions:

1. Join both datasets such that you get a merged dataset with 21 unique fields. You need to determine how to join the dataset.
2. Once you create the merged dataset, you need to convert fields to the following types, since all fields were read is as string:
    * pickup_datetime and dropoff_datetime must be TIMESTAMP
    * passenger_count and rate_code must be INT
    * all other numeric fields must be FLOAT
    * the remaining fields stay as STRING
3. Save your merged and converted dataset to your own S3 bucket in parquet format.

You are welcome to add as many cells as you need below up until the next section. **You must include comments in your code.**

In [4]:
import findspark
findspark.init()
from pyspark import SparkContext
sc=SparkContext()

In [5]:
sc

<SparkContext master=yarn appName=pyspark-shell>

In [8]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.getOrCreate()

In [9]:
spark

**Import Data files**

In [17]:
trip = spark.read.option("header", "true")\
  .parquet("s3://bigdatateaching/nyctaxi-2013/parquet/trip")

In [22]:
fare = spark.read.option("header", "true")\
  .parquet("s3://bigdatateaching/nyctaxi-2013/parquet/fare")

**Data Overview**

In [18]:
trip.describe()

DataFrame[summary: string, medallion: string, hack_license: string, vendor_id: string, rate_code: string, store_and_fwd_flag: string, pickup_datetime: string, dropoff_datetime: string, passenger_count: string, trip_time_in_secs: string, trip_distance: string, pickup_longitude: string, pickup_latitude: string, dropoff_longitude: string, dropoff_latitude: string]

In [19]:
trip.printSchema()

root
 |-- medallion: string (nullable = true)
 |-- hack_license: string (nullable = true)
 |-- vendor_id: string (nullable = true)
 |-- rate_code: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- pickup_datetime: string (nullable = true)
 |-- dropoff_datetime: string (nullable = true)
 |-- passenger_count: string (nullable = true)
 |-- trip_time_in_secs: string (nullable = true)
 |-- trip_distance: string (nullable = true)
 |-- pickup_longitude: string (nullable = true)
 |-- pickup_latitude: string (nullable = true)
 |-- dropoff_longitude: string (nullable = true)
 |-- dropoff_latitude: string (nullable = true)



In [23]:
trip.show(3)

+--------------------+--------------------+---------+---------+------------------+-------------------+-------------------+---------------+-----------------+-------------+----------------+---------------+-----------------+----------------+
|           medallion|        hack_license|vendor_id|rate_code|store_and_fwd_flag|    pickup_datetime|   dropoff_datetime|passenger_count|trip_time_in_secs|trip_distance|pickup_longitude|pickup_latitude|dropoff_longitude|dropoff_latitude|
+--------------------+--------------------+---------+---------+------------------+-------------------+-------------------+---------------+-----------------+-------------+----------------+---------------+-----------------+----------------+
|0FBCA8B7F51A9684E...|26AB68D4B482471A2...|      VTS|        1|              null|2013-03-15 07:42:00|2013-03-15 08:05:00|              1|             1380|         3.72|      -73.953934|      40.775021|       -73.993401|       40.747349|
|76B07812E69C59D13...|A9B1CD75F0D5A3205...| 

In [24]:
fare.describe()

DataFrame[summary: string, medallion: string, hack_license: string, vendor_id: string, pickup_datetime: string, payment_type: string, fare_amount: string, surcharge: string, mta_tax: string, tip_amount: string, tolls_amount: string, total_amount: string]

In [25]:
fare.printSchema()

root
 |-- medallion: string (nullable = true)
 |-- hack_license: string (nullable = true)
 |-- vendor_id: string (nullable = true)
 |-- pickup_datetime: string (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- fare_amount: string (nullable = true)
 |-- surcharge: string (nullable = true)
 |-- mta_tax: string (nullable = true)
 |-- tip_amount: string (nullable = true)
 |-- tolls_amount: string (nullable = true)
 |-- total_amount: string (nullable = true)



In [26]:
fare.show(3)

+--------------------+--------------------+---------+-------------------+------------+-----------+---------+-------+----------+------------+------------+
|           medallion|        hack_license|vendor_id|    pickup_datetime|payment_type|fare_amount|surcharge|mta_tax|tip_amount|tolls_amount|total_amount|
+--------------------+--------------------+---------+-------------------+------------+-----------+---------+-------+----------+------------+------------+
|180A298786862EA12...|BD9F6B0AED17B9CAB...|      VTS|2013-11-26 09:20:00|         CRD|         14|        0|    0.5|         1|           0|        15.5|
|F7F305E5D26B6A1A3...|52C8902960DA72824...|      VTS|2013-11-26 17:27:00|         CSH|       19.5|        1|    0.5|         0|           0|          21|
|64996CAE3371608EC...|8099958C7DA1A5601...|      VTS|2013-11-26 15:14:00|         CRD|          6|        0|    0.5|       1.2|           0|         7.7|
+--------------------+--------------------+---------+-------------------+---

**Merge Dataset**

In [27]:
# Create two tables called 'trip_table' and 'fare_table'
trip.createOrReplaceTempView("trip_table")
fare.createOrReplaceTempView("fare_table")

In [28]:
# Join two tables by the variables 'medallion','hack_license' and 'pickup_datetime'
# To make sure the 'fare' matches the same 'trip'
nyctaxi = spark.sql("""
SELECT
t.medallion              
, t.hack_license           
, t.vendor_id              
, t.pickup_datetime        
, f.payment_type           
, f.fare_amount            
, f.surcharge              
, f.mta_tax             
, f.tip_amount            
, f.tolls_amount           
, f.total_amount         
, t.rate_code              
, t.store_and_fwd_flag     
, t.dropoff_datetime       
, t.passenger_count     
, t.trip_time_in_secs      
, t.trip_distance          
, t.pickup_longitude       
, t.pickup_latitude        
, t.dropoff_longitude      
, t.dropoff_latitude      
FROM trip_table t
JOIN fare_table f 
ON
(t.medallion = f.medallion 
and t.hack_license = f.hack_license 
and t.pickup_datetime = f.pickup_datetime)
""").cache()

In [29]:
nyctaxi.printSchema()

root
 |-- medallion: string (nullable = true)
 |-- hack_license: string (nullable = true)
 |-- vendor_id: string (nullable = true)
 |-- pickup_datetime: string (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- fare_amount: string (nullable = true)
 |-- surcharge: string (nullable = true)
 |-- mta_tax: string (nullable = true)
 |-- tip_amount: string (nullable = true)
 |-- tolls_amount: string (nullable = true)
 |-- total_amount: string (nullable = true)
 |-- rate_code: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- dropoff_datetime: string (nullable = true)
 |-- passenger_count: string (nullable = true)
 |-- trip_time_in_secs: string (nullable = true)
 |-- trip_distance: string (nullable = true)
 |-- pickup_longitude: string (nullable = true)
 |-- pickup_latitude: string (nullable = true)
 |-- dropoff_longitude: string (nullable = true)
 |-- dropoff_latitude: string (nullable = true)



**Convert Data Type**

As you can see from printing the nyctaxi schema, all of the fields were loaded as strings. This will not work for modeling purposes. You need to convert some of the fields to other types.

In [32]:
colname_float = ["trip_distance","pickup_latitude","pickup_longitude","dropoff_latitude","dropoff_longitude","fare_amount","surcharge","mta_tax","tip_amount","tolls_amount","total_amount"]
colname_timestamp = ["pickup_datetime","dropoff_datetime"]

nyctaxi_converted = nyctaxi.withColumn("passenger_count", nyctaxi["passenger_count"].cast("integer"))
nyctaxi_converted = nyctaxi_converted.withColumn("trip_time_in_secs", nyctaxi["trip_time_in_secs"].cast("integer"))


for i in colname_float:
    nyctaxi_converted = nyctaxi_converted.withColumn(i, nyctaxi[i].cast("float"))
    
for i in colname_timestamp:
    nyctaxi_converted = nyctaxi_converted.withColumn(i, nyctaxi[i].cast("timestamp"))

nyctaxi_converted.printSchema()

root
 |-- medallion: string (nullable = true)
 |-- hack_license: string (nullable = true)
 |-- vendor_id: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- fare_amount: float (nullable = true)
 |-- surcharge: float (nullable = true)
 |-- mta_tax: float (nullable = true)
 |-- tip_amount: float (nullable = true)
 |-- tolls_amount: float (nullable = true)
 |-- total_amount: float (nullable = true)
 |-- rate_code: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_time_in_secs: integer (nullable = true)
 |-- trip_distance: float (nullable = true)
 |-- pickup_longitude: float (nullable = true)
 |-- pickup_latitude: float (nullable = true)
 |-- dropoff_longitude: float (nullable = true)
 |-- dropoff_latitude: float (nullable = true)



## In the following cells, please provide the requested code and output. Do not change the order and/or structure of the cells.

In the next cell, provide the code that saves your merged dataset to your S3 bucket.

In [33]:
nyctaxi_converted.write.parquet("s3://502hw4//merged_data",mode="overwrite")

In the next cell, print the schema of your merged dataset.

In [34]:
nyctaxi_converted.printSchema()

root
 |-- medallion: string (nullable = true)
 |-- hack_license: string (nullable = true)
 |-- vendor_id: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- fare_amount: float (nullable = true)
 |-- surcharge: float (nullable = true)
 |-- mta_tax: float (nullable = true)
 |-- tip_amount: float (nullable = true)
 |-- tolls_amount: float (nullable = true)
 |-- total_amount: float (nullable = true)
 |-- rate_code: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_time_in_secs: integer (nullable = true)
 |-- trip_distance: float (nullable = true)
 |-- pickup_longitude: float (nullable = true)
 |-- pickup_latitude: float (nullable = true)
 |-- dropoff_longitude: float (nullable = true)
 |-- dropoff_latitude: float (nullable = true)



In the next cell, print the first 10 records of your merged dataset.

In [35]:
nyctaxi_converted.show(10)

+--------------------+--------------------+---------+-------------------+------------+-----------+---------+-------+----------+------------+------------+---------+------------------+-------------------+---------------+-----------------+-------------+----------------+---------------+-----------------+----------------+
|           medallion|        hack_license|vendor_id|    pickup_datetime|payment_type|fare_amount|surcharge|mta_tax|tip_amount|tolls_amount|total_amount|rate_code|store_and_fwd_flag|   dropoff_datetime|passenger_count|trip_time_in_secs|trip_distance|pickup_longitude|pickup_latitude|dropoff_longitude|dropoff_latitude|
+--------------------+--------------------+---------+-------------------+------------+-----------+---------+-------+----------+------------+------------+---------+------------------+-------------------+---------------+-----------------+-------------+----------------+---------------+-----------------+----------------+
|00005007A9F30E289...|0649DA10C83DE7C6A...|

In the next cell, print the row count of your merged dataset.

In [36]:
print("Number of records: " + str(nyctaxi_converted.count()))

Number of records: 173185091
